In [1]:
import os
import sys
import pandas as pd
import numpy as np
module_path = os.path.abspath(os.path.join('../../data/..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%matplotlib inline

In [2]:
businessPrepped = pd.read_pickle('../../data/prep/businessPrepped.pkl')

In [3]:
businessPrepped.head()

,business_id,name,city,stars,review_count,is_open,postal_code,categories,checkins,tipcount
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",Ahwatukee,4.0,22,1,85044,Dentists;General Dentistry;Health & Medical;Or...,39.0,5.0
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",McMurray,3.0,11,1,15317,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...,15.0,1.0
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",Phoenix,1.5,18,1,85017,Departments of Motor Vehicles;Public Services ...,6.0,0.0
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",Tempe,3.0,9,0,85282,Sporting Goods;Shopping,120.0,3.0
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",Cuyahoga Falls,3.5,116,1,44221,American (New);Nightlife;Bars;Sandwiches;Ameri...,263.0,17.0


In [4]:

#Returns NaN for closed businesses, and the current star rating for open businesses.
def openStars(row):
    if row['is_open'] == 0:
       return np.nan
    return row['stars']

def closedStars(row):
    if row['is_open'] == 1:
       return np.nan
    return row['stars']


In [5]:
#businessPrepped['stars'] = businessPrepped['stars'].astype(float)
businessPrepped['open_stars'] = businessPrepped.apply(lambda row : openStars(row), axis=1)
businessPrepped['closed_stars'] = businessPrepped.apply(lambda row : closedStars(row), axis=1)
businessPrepped.head()

,business_id,name,city,stars,review_count,is_open,postal_code,categories,checkins,tipcount,open_stars,closed_stars
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",Ahwatukee,4.0,22,1,85044,Dentists;General Dentistry;Health & Medical;Or...,39.0,5.0,4.0,NaN
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",McMurray,3.0,11,1,15317,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...,15.0,1.0,3.0,NaN
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",Phoenix,1.5,18,1,85017,Departments of Motor Vehicles;Public Services ...,6.0,0.0,1.5,NaN
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",Tempe,3.0,9,0,85282,Sporting Goods;Shopping,120.0,3.0,NaN,3.0
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",Cuyahoga Falls,3.5,116,1,44221,American (New);Nightlife;Bars;Sandwiches;Ameri...,263.0,17.0,3.5,NaN


In [6]:

#Returns "" for closed businesses, and the current current category for open businesses.
def openCategories(row):
    if row['is_open'] == 0:
       return ""
    return row['categories']

def closedCategories(row):
    if row['is_open'] == 1:
       return ""
    return row['categories']



In [7]:
businessPrepped['closed_categories'] = businessPrepped.apply(lambda row : closedCategories(row), axis=1)
businessPrepped['open_categories'] = businessPrepped.apply(lambda row : openCategories(row), axis=1)
businessPrepped.head()

,business_id,name,city,stars,review_count,is_open,postal_code,categories,checkins,tipcount,open_stars,closed_stars,closed_categories,open_categories
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",Ahwatukee,4.0,22,1,85044,Dentists;General Dentistry;Health & Medical;Or...,39.0,5.0,4.0,NaN,,Dentists;General Dentistry;Health & Medical;Or...
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",McMurray,3.0,11,1,15317,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...,15.0,1.0,3.0,NaN,,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",Phoenix,1.5,18,1,85017,Departments of Motor Vehicles;Public Services ...,6.0,0.0,1.5,NaN,,Departments of Motor Vehicles;Public Services ...
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",Tempe,3.0,9,0,85282,Sporting Goods;Shopping,120.0,3.0,NaN,3.0,Sporting Goods;Shopping,
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",Cuyahoga Falls,3.5,116,1,44221,American (New);Nightlife;Bars;Sandwiches;Ameri...,263.0,17.0,3.5,NaN,,American (New);Nightlife;Bars;Sandwiches;Ameri...


Let's try create area profiles; #businesses, #open, #reviews, avg. stars, avg. starsOpen,category dictionary

starsOpen : Average Stars of Open Businesses in an area.

In [8]:
def categoryAgg(categoryColumn):
    areaCategories= {}
    for value in categoryColumn:
        categoryList = value.lower().split(";")
        for category in categoryList:
            if category in areaCategories:
                areaCategories[category] += 1
            elif category != "":
                areaCategories[category] = 1
    return areaCategories

In [9]:
#Operates on 'name' column and returns a dictionary counting the number of different chains.
def chainAgg(chainColumn):
    #Self-determined list of chains based on their appearances in dataset.
    chainList = ['"Starbucks"', '"McDonald"\'s','"Subway"', '"Pizza Hut"','"Taco Bell"','"Burger King"','"Walgreens"','"Wendy\'s"',\
                '"The UPS Store"', '"Tim Hortons"']
    areaChains= {}
    for chain in chainColumn:
        if chain in chainList:
            if chain in areaChains:
                areaChains[chain] += 1
            else :
                areaChains[chain] = 1
    return areaChains

In [10]:
#make second stars column for std. deviation of ratings
businessPrepped['std.dev_open_stars'] = businessPrepped['open_stars']

zipProfileGroups = businessPrepped.groupby('postal_code').agg({'business_id':'count', 'is_open':sum, 'review_count':sum,\
                                                               'checkins':sum, 'tipcount':sum,\
                                                               'open_stars':'mean','std.dev_open_stars':"std" ,\
                                                               'closed_stars':'mean', 'open_categories':categoryAgg, \
                                              'closed_categories':categoryAgg, 'name':chainAgg})

In [11]:
zipProfileDf = pd.DataFrame(zipProfileGroups)

In [14]:
zipProfileDf['%closed'] = (zipProfileDf['business_id'] - zipProfileDf['is_open']) / zipProfileDf['business_id']
#Rearrange order of Columns so %closed next to is open
zipProfileDf = zipProfileDf[['business_id','is_open','%closed','review_count','tipcount','checkins','open_stars','std.dev_open_stars','closed_stars',\
              'open_categories', 'closed_categories', 'name']]
zipProfileDf[zipProfileDf['business_id'] > 20].sort_values('%closed', ascending = False).head(10)

,business_id,is_open,%closed,review_count,tipcount,checkins,open_stars,std.dev_open_stars,closed_stars,open_categories,closed_categories,name
postal_code,,,,,,,,,,,,
15289,23,7,0.695652,832,137.0,2085.0,3.642857,0.899735,3.500000,"{'museums': 1, 'colleges & universities': 2, '...","{'sandwiches': 1, 'restaurants': 13, 'bakeries...",{}
85073,81,38,0.530864,1679,572.0,10915.0,3.526316,0.907531,3.127907,"{'farmers market': 1, 'food': 8, 'coffee & tea...","{'restaurants': 27, 'buffets': 1, 'chinese': 1...","{'""Starbucks""': 1, '""Burger King""': 1}"
89136,107,56,0.476636,3816,822.0,9160.0,3.517857,0.852612,3.284314,"{'shopping': 9, 'cafes': 1, 'restaurants': 14,...","{'restaurants': 30, 'japanese': 3, 'nightlife'...",{}
44178,21,12,0.428571,560,139.0,2407.0,3.916667,0.733402,3.444444,"{'parks': 2, 'active life': 2, 'restaurants': ...","{'home services': 1, 'professional services': ...",{}
28263,41,24,0.414634,1211,273.0,3578.0,3.437500,0.711894,3.352941,"{'delis': 1, 'sandwiches': 2, 'restaurants': 1...","{'food': 2, 'farmers market': 1, 'japanese': 3...",{}
M4V,68,44,0.352941,1102,127.0,1350.0,3.318182,1.045983,3.520833,"{'auto detailing': 1, 'auto repair': 2, 'autom...","{'food': 4, 'coffee & tea': 2, 'restaurants': ...",{}
M4Y,557,369,0.337522,16737,2582.0,31370.0,3.466125,0.869668,3.382979,"{'chicken wings': 7, 'fast food': 9, 'restaura...","{'restaurants': 107, 'greek': 2, 'fashion': 7,...","{'""Starbucks""': 5, '""Tim Hortons""': 1, '""The U..."
M6G,598,418,0.301003,17569,2557.0,28968.0,3.672249,0.863735,3.611111,"{'venues & event spaces': 4, 'dance clubs': 5,...","{'juice bars & smoothies': 2, 'food': 30, 'pub...","{'""The UPS Store""': 1, '""Subway""': 2, '""Starbu..."
M6J,737,519,0.295794,25414,3395.0,39381.0,3.826590,0.781535,3.697248,"{'dance clubs': 8, 'arts & entertainment': 26,...","{'himalayan/nepalese': 1, 'restaurants': 120, ...","{'""Starbucks""': 3, '""Tim Hortons""': 2, '""Subwa..."


In [15]:
zipProfileDf.to_pickle('../../data/analysis/ZIPprofiles.pkl')